### 적합도 함수 = 100 - ( 비용 + 승객의 이동시간 )

### 승객의 이동시간
- 승객이 해당 구간 이용시간 + 환승객의 이동 시간
    * 유입 승객수 : 순유입수 + 환승유입 승객수
    * 유출 승객수 : 순유출 + 환승유출 승객수
    
    * 이용 시간 : 전체 승객수 * (전구간 도착 시간 ~ 현 구간 도착 신간)
    * 이동 시간 : 환승유입 승객수 * 전 구간 이동 시간 + 환승 유출 승객수 * 현 구간 이동 시간 - 환승역이 두개 이상일 경우 평균으로 계산
    * 역에서의 이동시간은 도착 시간과 환승 이동 시간을 고려한 전역(현역)에서의 탑승 편의 직전(후) 편을 시간표를 이용하여 검색후 계산
    
- 알고리즘
    * 객체 데이터 구조 : 도착시간 | 구간1 속도 | 구간2 속도 | 구간3 속도 ...
    * 기준 데이터 : 각 역의 열차 시간표, 구간의 거리, 순유입/순유출/환승유입/환승유출 승객수
    * 1. 객체와 기준 데이터를 이용하여 각역의 도착 시간을 계산
    * 2. 각역에서의 환승 유입편 시간, 환승 유출편 찾기
    * 3. 각역에서의 cost 계산

In [1]:
import json

In [2]:
test_data = ['07:11:00',30,40,35,44,29,32,39,40,28,35,30]  # 첫 역 도착 시간, 구간 속도1, 2
time_table = {
    ## 역번호 : [라인, 환승시간(분), 편승표... ] - 역번호는 반드시 순서있게
    1:[
        ['6', 5,'07:03:00','07:09:00','07:15:00','07:20:00'],
    ],
    2:[
        ['2', 5,'07:00:00','07:03:00','07:06:00','07:12:00','07:15:00','07:19:00','07:25:00'],
    ],
    3:[
        ['5', 5,'07:02:00','07:07:00','07:12:00','07:22:00','07:27:00','07:31:00'],
    ]
}
port_table = { # 역거리, 순유입, 순유출, 환승유입, 환승유출
    1:(0,100,0,0,0),
    2:(2.6,100,30,0,0),
    3:(4.3,100,40,0,0),
    4:(3.5,100,30,0,0),
    5:(10.4,100,50,0,0),
    6:(4.7,100,20,0,0),
    7:(5.5,100,30,0,0),
    8:(6.6,100,50,0,0),
    9:(10.9,100,60,0,0),
    10:(3.4,100,70,0,0),
    11:(2.8,100,80,0,0),
    12:(3.3,100,90,0,0),
}

In [3]:
## 저장한 타임 테이블 json 파일을 이용해 기준 정보 생성
with open('time_table.json','r',encoding='utf-8') as f:
    time_table = json.load(f)
with open('port_table.json','r',encoding='utf-8') as f:
    port_table = json.load(f)

In [4]:
time_table.keys()

dict_keys(['7', '8', '9', '10', '11', '12'])

In [5]:
time_table['11']

[['5호선',
  4,
  '05:38:20',
  '05:49:20',
  '05:59:50',
  '06:09:00',
  '06:16:40',
  '06:22:40',
  '06:28:10',
  '06:34:10',
  '06:40:10',
  '06:46:10',
  '06:52:10',
  '06:58:10',
  '07:04:10',
  '07:09:10',
  '07:14:10',
  '07:19:10',
  '07:24:10',
  '07:29:10',
  '07:34:00',
  '07:38:00',
  '07:42:00',
  '07:46:00',
  '07:50:00',
  '07:54:00',
  '07:58:00',
  '08:01:50',
  '08:04:50',
  '08:07:50',
  '08:10:50',
  '08:13:50',
  '08:16:50',
  '08:19:50',
  '08:22:50',
  '08:26:10',
  '08:28:40',
  '08:31:10',
  '08:33:40',
  '08:36:10',
  '08:38:40',
  '08:41:10',
  '08:43:40',
  '08:46:10',
  '08:48:40',
  '08:51:10',
  '08:53:40',
  '08:56:10',
  '08:58:40',
  '09:01:40',
  '09:04:40',
  '09:07:40',
  '09:10:40',
  '09:13:40',
  '09:16:40',
  '09:19:50',
  '09:23:50',
  '09:28:00',
  '09:33:00',
  '09:38:00',
  '09:43:00',
  '09:48:00',
  '09:53:00',
  '09:58:00',
  '10:03:10',
  '10:09:10',
  '10:15:10',
  '10:21:10',
  '10:27:10',
  '10:33:10',
  '10:39:10',
  '10:45:10',
  '10:

In [6]:
def cal_move_time(d,s): ## 거리와 시간으로 소요 시간 계산
    return (d / s) * 60 * 60

def str_to_time(t): ## 문자열 초로 변환
    return int(t[:2])*60*60+int(t[3:5])*60+int(t[6:])

def time_to_str(t): ## 초를 문자열로 변환
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return "%02d:%02d:%02d" % (h, m, s)

In [7]:
## 각 역의 도착 시간 계산
def cal_arr_time_list(obj):
    train_time_list = [str_to_time(obj[0])] ## 첫역 도착 시간
    for i,port in enumerate(sorted(port_table.keys())[1:]):
        d = port_table[port][0] ## 거리
        s = test_data[i+1] ## 속도
        move_time = cal_move_time(d,s) ## 이동 시간
        train_time_list.append(train_time_list[i] + move_time)
    return [time_to_str(t) for t in train_time_list]

In [8]:
## 열차 도착 시간 계산
cal_arr_time_list(test_data)

['07:11:00',
 '07:17:48',
 '07:22:00',
 '07:27:39',
 '07:31:12',
 '07:40:05',
 '07:46:39',
 '08:02:39',
 '08:09:42',
 '08:21:29',
 '08:32:48',
 '08:54:36']

In [9]:
## 시간 문자열, 분 인터벌, 시간 문자열 리스트를 이용하여 해당 시간보다 인터벌 뒤(앞)의 시간 검색 하여 
def between_times(str_time,interval,times,after=False):
    int_time = str_to_time(str_time) ## 초단위로 변환
    int_times = [str_to_time(t) for t in times]
    if after:
        #print([t-int_time-interval*60 for t in int_times if t-int_time-interval*60 > 0])
        return [t-int_time-interval*60 for t in int_times if t-int_time-interval*60 > 0][0]+interval*60
    else:
        #print([int_time-t-interval*60 for t in int_times if int_time-t-interval*60 > 0])
        return [int_time-t-interval*60 for t in int_times if int_time-t-interval*60 > 0][-1]+interval*60

In [10]:
between_times('07:18:00',5,time_table['11'][0][2:])

530

In [11]:
between_times('07:18:00',5,time_table['11'][0][2:],after=True)

370

In [12]:
## 환승 유입 유출 소요 시간 검색
def cal_trans_times(obj):
    arr_time_list = cal_arr_time_list(obj) ## 각 역 도착 시간 리스트, 인덱스+1이 역의 키번호
    transin_time_list = []
    transout_time_list = []

    for port_key,arr_time in enumerate(arr_time_list):
        ## 환승 정보 체크하여 없으면 환승 시간 0
        if str(port_key+1) in time_table.keys():## 역 코드는 1부터?
            table = time_table[str(port_key+1)][0][1:] ## 환승선이 하나라고 가정하고 처리, 이후 여러개인 경우 까지 계산
            ## 각 역의 도착 시간과 타임테이블의 이동시간, 시간표를 비교하여 0보다 큰값의 첫번째 값 계산
            #print(port_key,arr_time,table,'\n\n')
            #print(table)
            if port_key == 0:
                ## 첫역은 환승 유출 고객 0
                transout_time_list.append(0)
                ## 환승 유입 고객 계산
                transin_time_list.append(between_times(arr_time,table[0],table[1:]))
            elif port_key == len(time_table)-1:
                ## 마지막 역은 환승 유입 고객 0
                transin_time_list.append(0)
                ## 환승 유출 고객 계산
                transout_time_list.append(between_times(arr_time,table[0],table[1:],after=True))
            else:
                ## 환승 유입 유출 고객 계산
                transin_time_list.append(between_times(arr_time,table[0],table[1:]))
                transout_time_list.append(between_times(arr_time,table[0],table[1:],after=True))
        else:
            transin_time_list.append(0)
            transout_time_list.append(0)
    
    train_time_list = [0]+[str_to_time(t[0])-str_to_time(t[1]) for t in zip(arr_time_list[1:],arr_time_list[:-1])]
    return train_time_list,transin_time_list,transout_time_list

In [13]:
## 역간소요시간, 환승 유입 유출 소요 시간 계산
move_times = cal_trans_times(test_data)
move_times

([0, 408, 252, 339, 213, 533, 394, 960, 423, 707, 679, 1308],
 [0, 0, 0, 0, 0, 0, 519, 319, 422, 509, 248, 816],
 [0, 0, 0, 0, 0, 0, 321, 281, 538, 511, 352, 804])

In [14]:
between_times('07:41:04',time_table['7'][0][1],time_table['7'][0][2:],after=False)

184

In [15]:
## 승객수 고려한 시간 계산
## 역간 이동 시간 : ( 이전역순유입 + 환승유입 ) * 소요시간
def cal_time_score(obj):
    move_times = cal_trans_times(obj) ## 역간 소요시간
    port_keys = sorted(port_table.keys())
    ## 승객 역간 소요 시간
    times1 = [t * (port_table[port_keys[i]][1] + port_table[port_keys[i]][3]) for i,t in enumerate(move_times[0][1:])]
    ## 환승 유입 승객 이동 및 대기 시간
    times2 = [t*p for t,p in zip(move_times[1],[port_table[k][3] for k in port_keys])][:-1]
    ## 환승 유출 승객 이동 및 대기 시간
    times3 = [t*p for t,p in zip(move_times[2],[port_table[k][4] for k in port_keys])][1:]
    return [sum(ts) for ts in zip(times1,times2,times3)]

In [16]:
score = cal_time_score(test_data)
score

[6539832,
 7662312,
 5648079,
 6503529,
 1630447,
 2457772,
 1931520,
 10201242,
 35799331,
 46780716,
 79209828]

In [17]:
sum(score)

204364608